In [23]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
from keras import regularizers
from keras import optimizers
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

%matplotlib inline

# 1. Read Data

In [24]:
def dummy_cat(df):
    cats = pd.DataFrame()
    for col in df.columns:
        if 'cat' in col:
            cats = pd.concat([cats,pd.get_dummies(df[col],prefix=col)],axis=1)
    return cats

In [25]:
def binary(df):
    cols = []
    for col in df.columns:
        if 'bin' in col:
            cols.append(col)

    return df[cols] 

In [27]:
def prepare(path):
    df = pd.read_csv(path,index_col='id')
    df = df.reset_index(drop=True)
    df = df[df['target']==0]
    
    bins = binary(df)
    cats = dummy_cat(df)
    
    return np.array(pd.concat([bins,cats],axis=1))

In [28]:
path = "~/Desktop/kaggle/Porto/train.csv"
X = prepare(path)

In [29]:
len(X)

573518

In [30]:
len(X[0])

201

In [41]:
def simple_model(X):
    train, test = train_test_split(X, test_size=0.4, random_state=42)
    valid, test = train_test_split(test,test_size=0.5, random_state=42)
    model = Sequential()
    model.add(Dense(80, input_dim=201, activation='relu'))
    model.add(Dense(201, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adadelta')
    callbacks = [EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=2, verbose=0)]

    model.fit(train, train, 
              epochs=500, 
              batch_size=1000, 
              validation_data=(valid,valid),
              callbacks=callbacks,
              verbose=False)
    return model.evaluate(test,test)

In [42]:
for i in [2000,5000,10000,20000,50000]:
    print i, simple_model(X[:i])


 9344/10000 [===========================>..] - ETA: 0s 0.0179371886939


In [43]:
for i in [100000,200000]:
    print i, simple_model(X[:i])

38560/40000 [===========================>..] - ETA: 0s